In [20]:
import os
import fnmatch
import cv2
import random
from datetime import datetime
import numpy as np

from sklearn.utils import shuffle

In [21]:
# # Load the TensorBoard notebook extension
# %load_ext tensorboard

In [22]:
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

from tensorflow.keras.optimizers import RMSprop, Adam, SGD

# Load images as numpy array

In [23]:
images = {"human": {}, "non-human": {}}
for path, sub_dirs, files in os.walk("../data/cropped/16x16"):
    for file in fnmatch.filter(files, "*.png"):
        file_path = os.path.join(path, file)
        img = file_path.split("/")[-1].split("_")[0]
        if "non-human" in file_path:
            images['non-human'].update({img: images['non-human'].get(img, []) + [cv2.imread(file_path)]})
        else:
            images['human'].update({img: images['human'].get(img, []) + [cv2.imread(file_path)]})

### Shape of image

In [24]:
in_shape = images['human'][list(images['human'].keys())[0]][0].shape
in_shape

(120, 120, 3)

# Hyperparameters

In [25]:
batch_size = 128
epochs = 10
verbose = 1
learning_rate = 0.001
# mom = 0.9 # momentum
input_shape = in_shape # (120, 120, 3) if 16x16 images selected

# Create cross validation set - leave one out

In [26]:
random.seed(1)
human_rand = random.sample([i for i in images['human'].keys() if i not in list(images['non-human'].keys())], k = images['non-human'].__len__())
dist = []
for idx, i in enumerate(list(images['non-human'].keys())):
    choices = [i, human_rand[idx]]
    rem = list(set(i for k, v in images.items() for i in v.keys() if i not in choices))
    dist.append([rem, choices])

### Now we have seven folds where in each fold, one image from radial and one from non-radial are left out for model evaluation

# Define the model architecture

In [27]:
cnn_model = Sequential([
    Conv2D(8, (3,3), activation='relu', input_shape=in_shape, name="CONVOLUTION_2D_1"),
    MaxPooling2D((2,2), name="MAXPOOL_2D_1"),
    Conv2D(16, (3,3), activation='relu', name='CONVOLUTION_2D_2'),
    MaxPooling2D((2,2), name="MAXPOOL_2D_2"),
    Conv2D(32, (3,3), activation='relu', name='CONVOLUTION_2D_3'),
    MaxPooling2D((2,2), name="MAXPOOL_2D_3"),
    Flatten(name="FLATTEN"),
    Dense(256, activation='relu', name="DENSE"),
    Dense(1, activation='sigmoid', name="SIGMOID")
], name='RADIAL_CNN')

# Compile the model

In [28]:
cnn_model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(lr=learning_rate),
    metrics=['accuracy']
)
cnn_model.summary()

Model: "RADIAL_CNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 CONVOLUTION_2D_1 (Conv2D)   (None, 118, 118, 8)       224       
                                                                 
 MAXPOOL_2D_1 (MaxPooling2D)  (None, 59, 59, 8)        0         
                                                                 
 CONVOLUTION_2D_2 (Conv2D)   (None, 57, 57, 16)        1168      
                                                                 
 MAXPOOL_2D_2 (MaxPooling2D)  (None, 28, 28, 16)       0         
                                                                 
 CONVOLUTION_2D_3 (Conv2D)   (None, 26, 26, 32)        4640      
                                                                 
 MAXPOOL_2D_3 (MaxPooling2D)  (None, 13, 13, 32)       0         
                                                                 
 FLATTEN (Flatten)           (None, 5408)              0

# Define the Keras TensorBoard callback

In [29]:
# logdir="logs/cnn_model/" + datetime.now().strftime("%y_%m_%d-%H_%M_%S_%s")
# tensorboard_callback = TensorBoard(log_dir=logdir)

In [30]:
acc_per_fold = []
loss_per_fold = []
history = []
fold_no = 1

for i in dist:
    inputs, outputs = {"train": [], "test": []}, {"train": [], "test": []}
    for keys, vals in images.items():
        bool_val = 0 if keys == "human" else 1
        for k, v in vals.items():
            var = 'train' if k in i[0] else 'test'
            inputs[var].extend(v)
            outputs[var].extend([bool_val] * len(v))
    for j in [inputs, outputs]:
        for k in j.keys():
            j[k] = np.stack(j[k], axis=0)
    inputs['train'], outputs['train'] = shuffle(inputs['train'], outputs['train'])
    inputs['test'], outputs['test'] = shuffle(inputs['test'], outputs['test'])
    inputs['train'], inputs['test'] = inputs['train']/255., inputs['test']/255.
    
    # Generate a print
    print('-----------------------------------------------------------------')
    print('Training for fold', fold_no, '...')

    # Fit data to model
    history.append(cnn_model.fit(inputs['train'], outputs['train'],
                                 batch_size=batch_size,
                                 epochs=epochs,
                                 verbose=verbose,
#                                  callbacks=[TensorBoard(log_dir=logdir+str(fold_no))],
                                 validation_data=(inputs['test'], outputs['test'])
                                )
                  )
#     Generate generalization metrics
    scores = cnn_model.evaluate(inputs['test'], outputs['test'], verbose=0)
    print('Score for fold {0}: {1} of {2}; {3} of {4}%'.format(fold_no, cnn_model.metrics_names[0], scores[0], cnn_model.metrics_names[1], scores[1]*100))
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no += 1
    
# == Provide average scores ==
print('=================================================================')
print('Score per fold')
for idx, i in enumerate(zip(loss_per_fold, acc_per_fold)):
    print('-----------------------------------------------------------------')
    print('> Fold {0} - Loss: {1:.6f} - Accuracy: {2:.6f}'.format(idx+1, i[0], i[1]))
print('=================================================================')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('=================================================================')

-----------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10
18/18 [==============================] - 9s 442ms/step - loss: 0.6501 - accuracy: 0.6773 - val_loss: 0.8147 - val_accuracy: 0.4820
Epoch 2/10
18/18 [==============================] - 8s 420ms/step - loss: 0.6168 - accuracy: 0.6936 - val_loss: 0.8994 - val_accuracy: 0.4820
Epoch 3/10
18/18 [==============================] - 8s 418ms/step - loss: 0.6149 - accuracy: 0.6936 - val_loss: 0.8597 - val_accuracy: 0.4820
Epoch 4/10
18/18 [==============================] - 7s 410ms/step - loss: 0.6133 - accuracy: 0.6936 - val_loss: 0.8123 - val_accuracy: 0.4820
Epoch 5/10
18/18 [==============================] - 7s 412ms/step - loss: 0.6135 - accuracy: 0.6936 - val_loss: 0.8861 - val_accuracy: 0.4820
Epoch 6/10
18/18 [==============================] - 7s 415ms/step - loss: 0.6139 - accuracy: 0.6936 - val_loss: 0.8599 - val_accuracy: 0.4820
Epoch 7/10
18/18 [==============================] - 7s 408

In [18]:
# %tensorboard --logdir logs